# Resilience against hardware failures

Scenario:  We have a cluster that partially consists of preemptible resources.  That is, we'll have to deal with workers suddenly being shut down during computation.  While demonstrated here with a `LocalCluster`, Dask's resilience against preempted ressources is most useful with, e.g., [Dask Kubernetes](https://kubernetes.dask.org/) or [Dask Jobqueue](https://jobqueue.dask.org).

Relevant docs: <http://distributed.dask.org/en/latest/resilience.html#hardware-failures>

## Increase resilience

Whenever a worker shuts down, the scheduler will increment the suspiciousness counter of _all_ tasks that were assigned (not necessarily computing) to the worker in question.  Whenever the suspiciousness of a task exceeds a certain threshold (3 by default), the task will be considered broken.  We want to compute many tasks on only a few workers with workers shutting down randomly.  So we expect the suspiciousness of all tasks to grow rapidly.  Let's increase the threshold:

In [1]:
import dask

dask.config.set({'distributed.scheduler.allowed-failures': 100});

## All other imports

In [2]:
from dask.distributed import Client, LocalCluster
from dask import bag as db
import os
import random
from time import sleep

## A cluster

In [3]:
cluster = LocalCluster(threads_per_worker=1, n_workers=4, memory_limit=400e6)
client = Client(cluster)
client

C:\Users\felip\AppData\Roaming\Python\Python310\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 50848 instead
  warnings.warn(
c:\Program Files\Python310\lib\contextlib.py:142: UserWarning: Creating scratch directories is taking a surprisingly long time. (1.35s) This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:50848/status,
Dashboard: http://127.0.0.1:50848/status,Workers: 4
Total threads: 4,Total memory: 1.49 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:50850,Workers: 4
Dashboard: http://127.0.0.1:50848/status,Total threads: 4
Started: Just now,Total memory: 1.49 GiB
Comm: tcp://127.0.0.1:50869,Total threads: 1
Dashboard: http://127.0.0.1:50870/status,Memory: 381.47 MiB
Nanny: tcp://127.0.0.1:50853,


## A simple workload

We'll multiply a range of numbers by two, add some sleep to simulate some real work, and then reduce the whole sequence of doubled numbers by summing them.

In [4]:
def multiply_by_two(x):
    sleep(0.02)
    return 2 * x

In [5]:
N = 400

x = db.from_sequence(range(N), npartitions=N // 2)

mults = x.map(multiply_by_two)

summed = mults.sum()

## Suddenly shutting down workers

Let's mark two worker process id's as non-preemptible.

In [6]:
all_current_workers = [w.pid for w in cluster.scheduler.workers.values()]
non_preemptible_workers = all_current_workers[:2]

In [7]:
def kill_a_worker():
    preemptible_workers = [
        w.pid for w in cluster.scheduler.workers.values()
        if w.pid not in non_preemptible_workers]
    if preemptible_workers:
        os.kill(random.choice(preemptible_workers), 15)

## Start the computation and keep shutting down workers while it's running

In [8]:
summed = client.compute(summed)

while not summed.done():
    kill_a_worker()
    sleep(3.0)

2024-07-05 15:03:01,199 - distributed.nanny - WARNING - Restarting worker
2024-07-05 15:03:04,180 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:50878' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('sum-part-5c69666a87d5d1a4d486d56e0ce64e71', 92), ('sum-part-5c69666a87d5d1a4d486d56e0ce64e71', 89), ('from_sequence-b2ab014b7dc5973a690fb7ca2c3348c9', 86), ('sum-aggregate-5c69666a87d5d1a4d486d56e0ce64e711', 3)} (stimulus_id='handle-worker-cleanup-1720202584.180003')
2024-07-05 15:03:04,210 - distributed.nanny - WARNING - Restarting worker
2024-07-05 15:03:07,179 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:50910' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('sum-aggregate-5c69666a87d5d1a4d486d56e0ce64e710', 10), ('sum-aggregate-5c69666a87d5d1a4d486d56e0ce64e711', 0), ('sum-part-5c69666a87d5d1a4d486d56e0ce64e71', 92)} (stimulus_id='handle-worker-cleanup-17

## Check if results match

In [9]:
print(f"`sum(range({N}))` on cluster: {summed.result()}\t(should be {N * (N-1)})")

`sum(range(400))` on cluster: 159600	(should be 159600)
